# Category Theory for Programmers

## 1.1

### Problems

1. Implement, as best as you can, the identity function in your favorite language (or the second favorite, if your favorite language happens to be Haskell).

In [1]:
# problem 1
def ident(x):
    return x

2. Implement the composition function in your favorite language. It takes two functions as arguments and returns a function that is their composition.

In [2]:
def compose(f, g):
    def composed(x):
        return f(g(x))
    return composed

3. Write a program that tries to test that your composition function respects identity.

In [3]:
def test_compose_ident():
    def square(x):
        return x * x

    for x in range(0, 10):
        assert compose(square, ident)(x) == compose(ident, square)(x) == square(x)

test_compose_ident()

4. Is the world-wide web a category in any sense? Are links morphisms?

* The web could be a category if there is a morphism whenever there is a path via links between two sites. Links themselves can't be the morphisms since there won't always be a self-link or a composition link.

5. Is Facebook a category, with people as objects and friendships as morphisms?

* Facebook is not a category with friendships being morphisms because they don't obey composition.

6. When is a directed graph a category?

* A directed graph is a category whenever every node has a self-loop, and the existence of a path from u -> v implies edge (u, v)

## 1.2

* Would like a correspondence between (set, function) and (type, haskell function), but there is a disanalogy due to non-halting computations. All types are then extended with _|_ ("Bottom"), to represent failing to terminate.
* Category of Haskell types and functions is `Hask`

### Methods of showing program correctness
* Operational semantics: state how a program would execute on an abstract machine
    * Basically just have to run the program on the idealized machine...
* Denotational semantics: Every programming construct is given a mathematical interpretation
    * Can give easy sales pitch for Haskell with `fact n = product [1..n]` being the definition of factorial
    * However, mapping I/O onto math seemed difficult until Eugenio Moggi found that it could be mapped to monads.

### Some Haskell types
* Void -> Empty set
    * Functions taking `Void` arguments can't be invoked
    * `absurd :: Void -> a` is uncallable and polymorphic in return type (from falsity follows anything)
* Types for singleton sets (e.g. C++ `void`)
    * Functions with void inputs may always be called, since the value just "is". Think of it as being called with a dummy value.
    * It seems that in this framing of functions, there's no way to discuss a function with no inputs other than `absurd`
    * Haskell uses () ("unit") for this value.
    * Pure functions of which map () to another type just pick out a value of the target type
    * Pure functions which return () do nothing
         * Because the formula `fInt:: Integer -> ()`, `fInt _ = ()` is the same for all input types, the function is called parametrically polymorphic
         * This function is called `unit`
* Bool
    * Two values
    * Pure functions from Bool just pick two values from the target type
    * Functions to Bool are called predicates

### Problems
1. Define a higher-order function (or a function object) memoize in your favorite language. This function takes a pure function f as an argument and returns a function that behaves almost exactly like f, except that it only calls the original function once for every argument, stores the result internally, and subsequently returns this stored result every time it’s called with the same argument. You can tell the memoized function from the original by watching its performance. For instance, try to memoize a function that takes a long time to evaluate. You’ll have to wait for the result the first time you call it, but on subsequent calls, with the same argument, you should get the result immediately.

In [4]:
from functools import wraps
def memoize(f):
    table = {}
    @wraps(f)
    def inner(x):
        if x not in table:
            table[x] = f(x)
        return table[x]
    return inner

2. Try to memoize a function from your standard library that you normally use to produce random numbers. Does it work?

In [5]:
import random
from functools import partial
randint_from_zero = partial(random.randint, 0)
memoized_randint = memoize(randint_from_zero)
print(memoized_randint(7))
print(memoized_randint(6))
print(memoized_randint(7))

0
0
0


Doesn't work

3. Most random number generators can be initialized with a seed. Implement a function that takes a seed, calls the random number generator with that seed, and returns the result. Memoize that function. Does it work?

In [6]:
def random_from_seed(seed):
    random.seed(seed)
    return random.random()

memoized_random = memoize(random_from_seed)
print(random_from_seed(73))
print(random_from_seed(62))
print(random_from_seed(73))
print(memoized_random(73))
print(memoized_random(62))
print(memoized_random(73))

0.27981547357298786
0.9279915658776743
0.27981547357298786
0.27981547357298786
0.9279915658776743
0.27981547357298786


Works

4. Which of these C++ functions are pure? Try to memoize them and observe what happens when you call them multiple times: memoized and not.
    1. The factorial function from the example in the text.
    2. ```c++
std::getchar()
```

    3. > ```c++
bool f() { 
    std::cout << "Hello!" << std::endl;
    return true; 
}
```

    4. > ```c++
int f(int x)
{
    static int y = 0;
    y += x;
    return y;
}```

    Only A is pure, assuming I'm remembering how static variables work in C++

5. How many different functions are there from Bool to Bool? Can you implement them all?

4

In [7]:
def always_true(b):
    return True

def always_false(b):
    return False

def ident(b):
    return b

def not_(b):
    return not b

6. Draw a picture of a category whose only objects are the types Void, () (unit), and Bool; with arrows corresponding to all possible functions between these types. Label the arrows with the names of the functions.

No

## 1.3 Categories Great and Small

* Empty category
    * Mostly important in relationship to other categories
* Free category
    * Given a directed graph, add all self loops, take each pair of composable edges (in edge meeting out edge), and add their composition to the graph
    * It's stated that this usually gives you infinitely many edges, so I assume that the composition of (u,v) and (v, w) is not the same as (u, t), (t, w)
    * In general "free construction" is the process of completing a structure by extending it with a minimum number of items to satisfy some laws

* Orders
    * Morphisms as relations
    * Pre-orders are categories:
        * $a \le a$ for all a
        * $a \le b,\:b \le c \implies a \le c$
    * Partial orders
        * Pre-order plus...
        * $a \le b,\:b \le a$ implies $a$ is $b$
    * Total orders
         * Partial order plus...
         * For all $a$ and $b$, either $a \le b$, or $b \le a$
         
    * Thin Categories: When a pre-order is viewed as a category, there is at most one morphism from any a to any b
    * The set of morphisms from $a$ to $b$ in category $C$ is call the hom-set and is written $C(a, b)$, or $\text{Hom}_C(a, b)$
    * All hom-sets either have 0 or 1 element in a thin category

### Monoids as sets
* Set with an associative binary operation that has an identity element.
* Examples: Integers with addition, Strings with concatenation
```
class Monoid m where
    mempty :: m
    mappend :: m -> m -> m
```

### Monoids as categories
* Consider operation of adding 5 to each natural number: 0 -> 5, 1 -> 6 etc
* Can compose adders: 5 adder composed with 7 adder is 12 adder
* `mappend` above is taking elements of the monoid set to the function corresponding to "adding" that element
* Monoids are single element categories with morphisms that compose correctly
* In the case of string concatenation, could consider right or left appenders, which lead to mirrored composition tables
* Can always extract a set from a single-object category, namely the hom-set of the single object
    * Equipped with the composition operator, it is a monoid
    * Technically the morphisms could be too numerous to be a set
        * A category in which all hom-sets are sets is called locally small

###Problems
1. Generate a free category from:
    1. A graph with one node and no edges
        * Identity arrow from node to itself
    2. A graph with one node and one (directed) edge (hint: this edge can be composed with itself)
        * Call edge $f$. Morphisms are identity on node, and $f$, $f∘f$, $f∘f∘f$ and so on.
    3. A graph with two nodes and a single arrow between them
        * Only add identity morphisms
    4. A graph with a single node and 26 arrows marked with the letters of the alphabet: a, b, c … z.
        * Morphisms are all strings including the empty string
2. What kind of order is this?
    1. A set of sets with the inclusion relation: A is included in B if every element of A is also an element of B.
        * Partial order
    2. C++ types with the following subtyping relation: T1 is a subtype of T2 if a pointer to T1 can be passed to a function that expects a pointer to T2 without triggering a compilation error.
        * Pre-order
3. Considering that Bool is a set of two values True and False, show that it forms two (set-theoretical) monoids with respect to, respectively, operator && (AND) and || (OR).
        * && and || are associative
        * True is &&'s identity, and False is ||'s identity
4. Represent the Bool monoid with the AND operator as a category: List the morphisms and their rules of composition.
    * Morphisms are `T = lambda x: True && x` and `F = lambda x: False && x`
    * $F∘F = F$
    * $T∘F = F$
    * $F∘T = F$
    * $T∘T = T$
5. Represent addition modulo 3 as a monoid category.
    * Morphisms are addition with 0, 1, and 2
    |   |0|1|2|
    |:--|-|-|-|
    |0|0|1|2|
    |1|1|2|0|
    |2|2|0|1|
    

## 1.4 Kleisli Categories

* The Writer Category
    * Have all functions additionally return a log message for example
    * Want an embellished version of `isEven :: int -> bool` which includes logging
        * Even though embellished function returns a pair `(bool, string)`, it's still considered an arrow between `int` and ` bool`
    * Can't use normal function composition since the extra strings aren't valid inputs to arrows coming from bool
    * New recipe for composition:
        * Execute first embellished func
        * Execute second func on first part of output of first func
        * Concatenate second outputs of both calls
        * Return tuple of first part of second func return, and concatenated string
    * Identity morphisms are identities, and append an empty string to the log
    * Can generalize this to any monoid for the "state" part
### Kleisli Categories
    * Writer is an example
    * A Kleisli category has (vaguely speaking):
        * Type as objects
        * Embellished functions as morphisms
### Problems
1. Construct the Kleisli category for partial functions (define composition and identity).

* The Writer Category
    * Have all functions additionally return a log message for example
    * Want an embellished version of `isEven :: int -> bool` which includes logging
        * Even though embellished function returns a pair `(bool, string)`, it's still considered an arrow between `int` and ` bool`
    * Can't use normal function composition since the extra strings aren't valid inputs to arrows coming from bool
    * New recipe for composition:
        * Execute first embellished func
        * Execute second func on first part of output of first func
        * Concatenate second outputs of both calls
        * Return tuple of first part of second func return, and concatenated string
    * Identity morphisms are identities, and append an empty string to the log
    * Can generalize this to any monoid for the "state" part
    
### Kleisli Categories
    * Writer is an example
    * A Kleisli category has (vaguely speaking):
        * Type as objects
        * Embellished functions as morphisms
        
### Problems
1. Construct the Kleisli category for partial functions (define composition and identity).
    * Composition of `f :: a -> b` and `g :: b -> c` is a function which:
        * Runs `f` on input
        * If output is not valid, returns `optional<c>{}`
        * Otherwise returns `g(f(input))`

2. Implement the embellished function safe_reciprocal that returns a valid reciprocal of its argument, if it’s different from zero.

In [8]:
class Optional:
    def __init__(self, value=None):
        self.value = value
        self.valid = self.value is not None

def safe_root(x):
    return Optional(x ** 0.5 if x >= 0 else None)

def safe_reciprocal(x):
    return Optional(1  / x if x != 0 else None)

3. Compose safe_root and safe_reciprocal to implement safe_root_reciprocal that calculates sqrt(1/x) whenever possible.

In [9]:
def compose_optional(f, g):
    def inner(x):
        out1 = g(x)
        return Optional() if not out1.valid else f(out1.value)
    return inner

safe_inv_sqrt = compose_optional(safe_root, safe_reciprocal)

In [10]:
print(safe_inv_sqrt(0).value)
print(safe_inv_sqrt(-1).value)
print(safe_inv_sqrt(2).value)
print(safe_inv_sqrt(-0.).value)
print(safe_inv_sqrt(1).value)

None
None
0.7071067811865476
None
1.0


## 1.5 Producs and coproducts

* In category theory, objects can only be distinguished from each other by their morphisms
* The universal construction for defining objects in terms of their relationships:
    * Choose a pattern, find all its occurrences in the category, and pick the "best fit"

### Initial object
* The simplest shape
* Has exactly one morphism going to every object
* This doesn't guarantee uniqueness, but does guarantee uniqueness up to isomorphism
* Examples
    * Initial element of a poset is its least element
    * In category of sets and functions, initial object is the empty set
    

### Terminal object
* Call object $a$ more terminal than $b$ if there is a morphism going from $b$ to $a$
* The *terminal object* is the object with exactly one morphism coming to it from any object in the category
* Unique up to isomorphism
* Examples
    * Largest element of poset
    * Unit in Haskell, void in C++

### Duality
* From category C, get the *opposite category* $C^{\text{op}}$ by:
    * Reversing all arrows
    * Modify composition:
        * $f::a\rightarrow b$ and $g::b\rightarrow c$ composing to $h:: a \rightarrow c$ with $g \circ f = h$ implies:
        * $f^{\text{op}}::b\rightarrow a$ and $g^{\text{op}}::c \rightarrow b$ compose to $h^{\text{op}}::c\rightarrow a$ with $h^{\text{op}} = f^{\text{op}}\circ g^{\text{op}}$
* Terminal object is initial object in opposite category

### Isomorphisms
* An isomorphism is an invertible morphism, or a pair of morphisms which are inverses of each other
    * $f \circ g = \text{id}$
    * $g \circ f = \text{id}$
* Initial and terminal objects are unique up to not just isomorphism, but *unique* isomorphism

### Products
* What pattern connects the product set with its constituents?
    * Two functions, the projections, from the product to each of the constituents
    * e.g. `fst :: (a, b) -> a`
* Object $c$ and morphisms connecting it with $a$ and $b$
* May be many such objects
    * Int could be considered for product of `Int` and `Bool`
        * `p :: Int -> Int`, `p x = x`
        * `q :: Int -> Bool`, `q _ = True`
    * `(Int, Int, Bool)` could be considered with the second elementy always being discarded
    * First is too small, second is too big
* A *product* of objects $a$ and $b$ is the object $c$ which has two projections such that for any other $c'$ having projections there is a unique $m$ from $c'$ to $c$ which factorizes those projections
* The factorizer produces the $m$ from two candidates:
    * `factorizer :: (c -> a) -> (c -> b) -> (c -> (a, b))`
    * `factorizer p q = \x -> (p x, q x)`
        

### Coproduct
* A coproduct of two objects $a$ and $b$ is the object $c$ equipped with two injections such that for any other object $c’$ equipped with two injections there is a unique morphism $m$ from $c$ to $c’$ that factorizes those injections.
* In category of sets, coproduct is *disjoint union*
    * Disjoint union contains every element of both sets, and two copies of everything in their intersection (by tagging which set an element came from)
* For types, coproduct is tagged union of two types
    * Me: Anything that has injections from types A and B also has a function from A or B "passing through" the tagged union
    * Defined in Haskell as `Either = Left a | Right b`
* Factorizer for coproduct:
    * `factorizer :: (a -> c) -> (b -> c) -> Either a b -> c`
    * `factorizer i j (Left a)  = i a`
    * `factorizer i j (Right b) = j b`
    * `factorizer` takes 3 arguments, the 3rd of which is the left or right from `Either`

### Asymmetry in category of sets
    * Products are very different from Coproducts, initial objects are very different from final objects
    * Therefore categorie of sets isn't symmetric with respect to arrow reversal

### Problems
1. Show that the terminal object is unique up to unique isomorphism.
    * Let $t$ and $t'$ be terminal objects
    * There is exactly one morphism $f$ from $t$ to $t'$, and exactly one $g$ from $t'$ to $t$
    * There is exactly one morphism from $t$ to itself, and from $t'$ to itself, which must be the identities
    * $f \circ g$ is a morphism from $t'$ to $t'$, and so must be the identity
    * $g \circ f$ is a morphism from $t$ to $t$, and so must be the identity
    * $f$ and $g$ are there for the unique isomorphisms between $t$ and $t'$ (unique b/c terminal requires exactly one morphism)
2. What is a product of two objects in a poset? Hint: Use the universal construction
    * The product of $a$ and $b$ is $\min(a, b)$:
        * $\min(a, b) \le a$ and $\min(a, b) \le b$
        * For any $c$ such that $c \le a$ and $c \le b$, $c \le \min(a, b)$ as well
3. What is a coproduct of two objects in a poset?
    * The coproduct of $a$ and $b$ is $\max(a, b)$:
        * $a \le \max(a, b)a$ and $b \le \max(a, b)$
        * For any $c$ such that $a \le c$ and $b \le c$, $\max(a, b) \le c$ as well
4. Implement the equivalent of Haskell Either as a generic type in your favorite language (other than Haskell).

In [11]:
class Either:
    def __init__(self, left=None, right=None):
        if not (left is None or right is None):
            raise ValueError
        self.left = left
        self.right = right

5. Show that Either is a “better” coproduct than int equipped with two injections:
```c++
int i(int n) { return n; }
int j(bool b) { return b? 0: 1; }
```
Answer:
```c++
int m(Either const & e) {
    if (e has int){
        return e.int;
    } else {
        return e.bool ? 0: 1;
    }
}
```

6. Continuing the previous problem: How would you argue that int with the two injections i and j cannot be “better” than Either? 
* A candidate $m'$ from `int` to `Either` would need map `0` to either `True` or `0`, meaning it can't simultaneously factorize both of `Either`'s injections

7. Still continuing: What about these injections?
```c++
int i(int n) { 
    if (n < 0) return n; 
    return n + 2;
}
int j(bool b) { return b? 0: 1; }```
* Had to look this up: `int` has a finite number of values, so the first one is _not_ an injection

8. Come up with an inferior candidate for a coproduct of int and bool that cannot be better than Either because it allows multiple acceptable morphisms from it to Either.
* `(bool, int, bool)`, with injections
* `i :: int -> (bool, int, bool)`
* `j = \x (False, x, False)`
* `i :: bool -> (bool, int, bool)`
* `i = \x (True, 0, x)`
* The behavior of the factorizing function `m` on inputs such as `(True, 17, True)` is not specified so it is not unique


## 1.6 Simple Algebraic Data Types
* **Set** is a symmetric monoidal category with respect to both products and coproducts
    * Unit of product is "unit": `()`
    * Identity of coproduct is `Void`
* Analogy between product and sum types, and products and sums
* Does multiplication by zero give zero?
    * Yes, `(Int, Void)` has no elements
* Distributive property
    * Numbers: a * (b + c) = a * b + a * c
    * Types: `(a, Either b c)` isomorphic to `Either (a, b) (a, c)`
* Two connected monoids like this are called a _semiring_)

### Problems
1. Show the isomorphism between `Maybe a and Either () a`
```Haskell
type Other = Either () Int
f :: Maybe Int -> Other
f x =
    case x of
        Nothing -> Left ()
        Just val -> Right val
```
```Haskell
g :: Other -> Maybe Int
g y =
    case y of
        Left () -> Nothing
        Right val -> Just val
```
2. Implement `Shape` in C++ or Java as an interface and create two classes: `Circle` and `Rect`. Implement area as a virtual function.

3. Add `circ` to your C++ or Java implementation. What parts of the original code did you have to touch?

4. Continuing further: Add a new shape, `Square`, to `Shape` and make all the necessary updates. What code did you have to touch in Haskell vs. C++ or Java? (Even if you’re not a Haskell programmer, the modifications should be pretty obvious.)

In [12]:
from abc import ABC, abstractmethod
import math

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass
    
    @abstractmethod
    def circ(self):
        pass
    
class Circle(Shape):
    def __init__(self, radius):
        super(Circle, self).__init__()
        self.radius = radius
    def area(self):
        return math.pi * radius * radius
    
    def circ(self):
        return 2 * self.radius * math.pi
    
class Rect(Shape):
    def __init__(self, height, width):
        super(Rect, self).__init__()
        self.height = height
        self.width = width
    def area(self):
        return self.height * self.width
    
    def circ(self):
        return 2 * (self.height + self.width)
    
class Square(Rect):
    def __init__(self, height):
        super(Square, self).__init__(self.height, self.height)
    def area(self):
        return self.height * self.width
    
    def circ(self):
        return 2 * (self.height + self.width)

5. Show that a + a = 2 * a holds for types (up to isomorphism). Remember that 2 corresponds to `Bool`, according to our translation table.

* The isomorphism is witnessed by the following:

```Haskell
f :: Either Int Int -> (Bool, Int)
f x = case x of
    Left val -> (False, val)
    Right val -> (True, val)
```
```Haskell
g :: (Bool, Int) -> Either Int Int
g y = case y of
    (False, val) -> Left val
    (True, val) -> Right val
```

## 1.7 Functors
* A functor is a mapping between categories
* Between categories $C$ and $D$, a Functor $F$ maps objects to objects
    * $F \: a$ in $D$ is image of object $a$ from $C$
* $F$ maps morphisms preserving connections
    * So if $f :: a \rightarrow b$ in $C$
    * $F\: f :: F\: a \rightarrow F\: b$ in $D$ 
* It also preserves composition
    * $h = g \circ f$ implies
    * $F\: h = F\: g \circ F \circ f$
* Also preserves identity
* Functors can merge objects and combine morphisms, but not "tear" a category

### Functors in Programming
* A Functor from a category to itself is called an endofunctor

#### The Maybe Functor
* `Maybe` maps from type `a` to type `Maybe a`
* Not a type, but a _type constructor_
* Only produces a type when given an argument
* Define `fmap` which lifts a function to act on `Maybe` values (just return `Nothing` when given `Nothing`)

### Equational Reasoning
* In Haskell functions are defined as equalities, so can always substitute one side for the other
* Example: Preserving identity

```Haskell
-- Prove:
fmap id = id
```
```Haskell
fmap id Nothing = Nothing -- definition of fmap
= id Nothing -- definition of id
```
```Haskell
fmap id (Just x) = Just (id x) -- definition of fmap
= Just x -- definition of id
= id (Just x) -- definition of id
```

### Typeclasses
```Haskell
class Functor f where
    fmap :: (a -> b) -> f a -> f b
```
* $f$ is a type variable in the above, but the compiler can tell that it must be a constructor due to its usage

### The List Functor
* `fmap` defined recursively to match recursive definition of `List`

### The Reader Functor
* Map a type `a` to the type of a function returning `a`
* Type signature of fmap:
    * `fmap :: (a -> b) -> (r -> a) -> (r -> b)`
    * i.e. Given a function `a->b` and a function `r->a`, produce a function `r->b`
    * Just compose:
        ```Haskell
        instance Functor ((->) r) where
            fmap f g = f . g
        ```

### Functors as Containers
* Can think of reader Functor as a container if you consider functions as (usually infinite) tables
* Think of objects of type generated by an endofunctor as containing a value or values of the type it was parameterized by
* It could also be a recipe for generating those values
* Doesn't matter if we can access the values, just want to manipulate them with functions
    * If can acess values, want to see results of manipulation
    * Otherwise just care that manipulations compose correctly and that identities work

### Functor Composition
* Functor composition is associative and every category has identity functor, so they have the properties required of morphisms
* Category of all categories doesn't exist but
* Category of _small_ categories (called **Cat**) exists
* Small categories are those in which objects form a set

### Problems

1. Can we turn the Maybe type constructor into a functor by defining:
`fmap _ _ = Nothing`
which ignores both of its arguments?
    * No, if you take an isomorphism between two types `a` and `b`, this will fail to produce an isomorphism between `Maybe a` and `Maybe b`.

2. Prove functor laws for the reader functor. Hint: it’s really simple.
* $(F\: g \circ f)(h) = g(f(h)) = (F\: g)(F\: f)(h)$

3. Implement the reader functor in your second favorite language

In [13]:
def reader_fmap(func):
    def inner(in_func):
        def more_inner(x):
            return func(in_func(x))
        return more_inner
    return inner
square = lambda x: x * x
reader_fmap(square)(int)('17')

289

4. Prove the functor laws for the list functor. Assume that the laws are true for the tail part of the list you’re applying it to (in other words, use induction).
    * Suppose the laws hold for all `Lists` of length $n$
    * Identity maps to identity:
        * `fmap id (Cons x t) = Cons (id x) (fmap id t)`
        * `= Cons x (fmap id t)` (def of id)
        * `= Cons x t` (inductive hyp.)
        * `= id (Cons x t) (def of id)
    * Composition:
        * `fmap (f . g) (Cons x t) = Cons ((f . g) x) (fmap (f . g) t)`
        * `= Cons ((f . g) x) (((fmap f) . (fmap g)) t)` (inductive hyp.)
        * `= fmap f (Cons (g x) (fmap g t))` (def of fmap)
        * `= ((fmap f) . (fmap . g)) (Cons x t)` (def of fmap)

## 1.8 Functoriality
### Bifunctors
* Functors of two arguments
    * From categories C and D, to E
    * Consider pairs of morphisms from the product category $C \times D$
        * Composition: $(f, g) \circ (f', g') = (f \circ f', g \circ g')$
        * Identity is easy
* Would like to think about each argument separately, but functoriality in each argument is not enough to prove joint functoriality
    * If joint functoriality fails, category is called _premonoidal_
* `first` and `second` in Haskell `Bifunctor`:
    * `first` by default applies `bimap` to one function and the identity
    * `second` applies `bimap` to the identity and another function

#### Product and Coproduct Bifunctors
* "If the product exists for any pair of objects, the mapping from the objects to the product is bifunctorial"
* In **Hask**, forming a pair of types from two types is a Bifunctor
    * `bimap` creates function which applies first function to first element, and second function to second element
* Coproduct is also a bifunctor by duality
    * `bimap` uses first function if `Either` is a `Left`, and second if it's a `Right`
* Monoidal category also requires that monoidal product be a bifunctor

#### Functorial Algebraic Data Types
* View ADTs as Functors built up from `Identity` and `Const` using products and sums
* `data Maybe a = Nothing | Just a` is isomorphic to `type Maybe a = Either (Const () a) (Identity a)`

### The Writer Functor
* `type Writer a = (a, String)`
    * Product type which is functorial in `a`
* Kleisli category defines composition and identity, but where is the `fmap` if it's related to a functor?
* Given composition `>=>` and identity (`return`), can come up with:
    * `fmap f = id >=> (\x -> return (f x))`
    * `id` is of type `Writer a -> Writer a` here
    * fish operator takes value of output, applies `f` to it, then converts it to a `Writer b` with `return`
* General enough that embellishment in a Kleisli category is always a functor (but not vice versa)
* If there is an implementation of `fmap` for a type constructor that preserves identity, it is unique

### Covariant and Contravariant Functors
* Consider `type Op r a = a -> r`
    * `fmap` type would be `(a -> b) -> (a -> r) -> (b -> r)`
    * Given `a->b` and `a -> r`, can't make a function which takes `b` and gives `r`
        * Would like to be able to invert the first argument, but can't
        * Can take the arrow from the opposite category
* $F :: C^{\text{op}} \rightarrow D$ maps $f^{\text{op}} :: a \rightarrow b$ to $F\: f^{\text{op}} :: F\:a \rightarrow F\:b$
* But $f^{\text{op}}$ corresponds to some $f$ in the original category $C$
* $F$ is a regular functor, but can define $G$ which maps from $C$ to $D$
* $G$ maps objects in the same way, but reverses morphisms before mapping them
* $G\: f :: (b \rightarrow a) \rightarrow (G\: a \rightarrow G\: b)$
    * This is a contravariant functor

### Profunctors
* `(->)` is contravariant in first arg, covariant in second
* If target category is **Set**, this is a _profunctor_
    * $C^{\text{op}} \times D \rightarrow \textbf{Set}$

### The Hom-Functor
* Above are just cases of general statement that mapping of two objects, `a` and `b`, to set of morphisms (the hom-set `C(a, b)` between them is a functor
    * $C^{\text{op}} \times C \rightarrow \textbf{Set}$
* What does it do to morphisms?
    * A morphism on $C^{\text{op}} \times C$ is a pair of morphisms $f:: a' \rightarrow a$ and $g::b\rightarrow b'$
    * The lifting is a function from $C(a, b)$ to $C(a', b')$
    * Pick any $h$ in $C(a, b)$ and assign to it $g \circ h \circ f$ which is an element of $C(a', b')$
* So hom-functor is a profunctor

### Problems
1. Show that `data Pair a b = Pair a b` is a bifunctor. For additional credit implement all three methods of Bifunctor and use equational reasoning to show that these definitions are compatible with the default implementations whenever they can be applied.
    * Identity:
    * `bimap id id (Pair x y) = Pair (id x) (id y)`
    * `= Pair x y`
    * `= id (Pair x y)`
    * Composition:
    * `bimap (f . g) (h . i) (Pair x y) = Pair ((f . g) x) ((h . i) y)`
    * `= bimap f h (Pair (g x) (i y))`
    * `= (bimap f h) (bimap g i) (Pair x y)`
    * Implementations
        ```Haskell
        bimap f g (Pair x y) = Pair (f x) (g y)
        first f (Pair x y) = Pair (f x) y
        second g (Pair x y) = Pair x (g y)
        ```
    * Proving they match seems boring
2. Show the isomorphism between the standard definition of Maybe and this desugaring:
`type Maybe' a = Either (Const () a) (Identity a)`
```Haskell
onetotwo :: Maybe a -> Maybe' a
onetotwo Nothing = Left (Const ())
onetotwo (Just x) = Right (Identity x)
```
```Haskell
twotoone :: Maybe' a -> Maybe a
twotoone (Left _) = Nothing
twotoone (Right (Identity x)) = Just x
```

3. Let’s try another data structure. I call it a PreList because it’s a precursor to a List. It replaces recursion with a type parameter b.
`data PreList a b = Nil | Cons a b`  
You could recover our earlier definition of a `List` by recursively applying `PreList` to itself (we’ll see how it’s done when we talk about fixed points).  
Show that `PreList` is an instance of `Bifunctor`.
    * `bimap f g (Nil) = Nil`
    * `bimap f g (Cons x y) = Cons (f x) (g y)`
    * Identity:
    * `bimap id id (Cons x y) = Cons (id x) (id y)`
    * `= id (Cons x y)`
    * Composition:
    * `bimap (f . g) (h . i) (Cons x y) = Cons ((f.g) x) (h.i)y)`
    * `= (bimap f h) (Cons (g x) (i y))`
    * `= (bimap f h) ((bimap g i) (Cons x y))`
    
4. Show that the following data types define bifunctors in a and b:  
    * `data K2 c a b = K2 c`
        * `bimap f g (K2 x) = (K2 x)`
        * Clearly respects identity
        * Maps everything to identity so everything composes
    * `data Fst a b = Fst a`
        * `bimap f g (Fst x) = Fst (f x)`
        * Clearly respects identity
        * Composition
        * `bimap (f . g) (h . i) (Fst x) = Fst ((f . g) x)`
        * `= (bimap f h) (Fst (g x))`
        * `= (bimap f h) ((bimap g i) (Fst x))`
5. Define a bifunctor in a language other than Haskell. Implement bimap for a generic pair in that language.

In [14]:
def bimap_pair(f, g, tup):
    x, y = tup
    return f(x), g(y)

6. Should `std::map` be considered a bifunctor or a profunctor in the two template arguments `Key` and `T`? How would you redesign this data type to make it so?
    * `bimap :: (a -> b) -> (c -> d) -> (Map<a><c> -> Map<b><d>)`
    * Can't work because keys can collide
    * `dimap :: (b -> a) -> (c -> d) -> (Map<a><c> -> Map<b>,d>)`
    * `dimap f g map` is a map where the `get` is:
    * `get key = g . original_get . f`
    * Probably doesn't work out of the box, so throw in some `Maybe`s

## 1.9 Function Types
* Function types are different from other types because each one is a set of morphisms between two objects in **Set**, i.e. a hom-set
* Not true of all categories, so when hom-sets don't live in the category they are called "external hom-sets"
* In some categories, can construct objects that represent hom-sets

### Universal construction
* Want to construct function type (internal hom-set) from scratch
* Pattern involves three objects: the function type ($z$), the argument type ($a$), and the result type ($b$)
    * These are connected by a pattern called _function application_ or _evaluation_
* Would like a pair consisting of one element $f \in z$ and one element $x \in a$, so we can get $f\: x \in b$, but we can't look inside objects
* Instead, consider product of $z$ and $a$, $z\times a$, and pick an arrow $g :: z \times a\rightarrow b$
* For ranking, say that $z$ with $g$ from $z\times a$ to $b$ is better than $z'$ with $g'$ if there is an $h :: z' \rightarrow z$ such that $g'$ factors through application of $g$.
* Problem is, $g$ originates from $z \times a$, so we need not just $h$, but $(h, \text{id})$ (which we can get from the product because it is a bifunctor which doesn't just combine objects, but morphisms as well)
    * $g' = g \circ (h \times \text{id})$

* Call the best object $a\Rightarrow b$
    * Comes with application, a morphism $\text{eval} :: (a\Rightarrow b \times a) \rightarrow b$
    * Object is the best if any other candidate can uniquely be maped to it such that its application $g$ factors through `eval`

### Currying
* Think of function candidate morphism $g$ as a function of two variables, $z$ and $a$, $g :: z \times a \rightarrow b$
* $g$ is a function from pairs of values
* Universal proprety tells us that for each $g$, there is a unique $h$ that maps $z$ to a function object $a\Rightarrow b$
    * $h :: z \rightarrow (a\Rightarrow b)$
* In **Set** this makes $h$ a higher order function, since it is taking in an argument of type $z$, and returning a function from $a$ to $b$
* This means universal construction connects functions of two variables and functions of one variable which return functions.
    * $h$ is the curried version of $g$
    * One to one because:
        * Universal construction requires that given any $g$ there is a unique $h$
        * Given $h$, can reconstruct two argument version as $g = \text{eval} \circ (h \times \text{id})$
* `curry :: ((a, b)->c) -> (a->b->c)` is the _factorizer_ for the universal constrcution of the function object
    * A factorizer produces the factorizing function from a candidate
    
### Exponentials
    * The function object (or internal hom-object) between objects $a$ and $b$ is called the exponential, $b^a$
    * To start with, was necessary to use product in universal internal hom-object construction
    * Given two finite types $a$ and $b$, number of functions from $a$ to $b$ is $|b|^{|a|}$

### Curry-Howard Isomorphism
* Correspondence between logic and algebraic data types
* `Void` and `()` correspond to false and true
* Product and sum types correspond to $\land$ and $\lor$
* Function types correspond to $\implies$
* Every ADT can then be interpreted as a proposition (it is true if the type is inhabited, and false if it isn't)

## 1.10 Natural Transformations
* Think of functors as embedding one category in another
* Natural transformations let compare different embeddings
* "Natural" to use existing connections so
    * Call transformation between functors $F$ and $G$ $\alpha$
    * $F\: a \rightarrow G\: a$ is called component of $\alpha$ at $a$
    * If there is no such morphism, there can be no natural transformation
* Morphisms also must be mapped: $F\: f$ to $G\: f$
    * $F\: f :: F\: a \rightarrow F\: b$
    * $G\: f :: G\: a \rightarrow G\: b$
    * Natural transformation $\alpha$ provides:
        * $\alpha_a :: F\: a \rightarrow G\: a$
        * $\alpha_b :: F\: b \rightarrow G\: b$
    * Require that two ways of getting from $F\: a$ to $G\: b$ are equal
        * $G\: f \circ \alpha_a = \alpha_b \circ F\: f$
* If morphisms $F\: f$ is invertible, naturality means:
    * $\alpha_b = (G\: f) \circ \alpha_a \circ {(F\: f)}^{-1}$
* Can also think of natural transformation as mapping objects to morphisms ($a \rightarrow \alpha_a$?),
* Or can think of it as mapping morphisms to commuting squares
* A natural transformation whose components are all isomorphisms is called a natural isomorphsism

### Polymorphic Functions
* To construct natural transformation, start with $a$. $F$ maps it to $F\: a$, $G$ maps it to $G\: a$
* `alpha_a :: F a -> G a`
* A natural transformation is a polymorphic function defined for all types $a$:
    * `alpha :: forall a . F a -> G a`
    * Normally write `alpha :: F a -> G a`
    * Family of functions parameterized by $a$
* In Haskell, `alpha :: F a -> G a` where $F$ and $G$ are functors automatically satisfies naturality condition:
    * $G\: f \circ \alpha_a = \alpha_b \circ F\: f$
    * `fmap_G f . alpha_a = alpha_b . fmap_f f`
    * Apparently this is true because of "theorems for free"
* Haskell functions must be defined uniformly for all types
* If think of functors as containers, natural transformations are repackagings of the contents into another container
* Natural transformations from or to `Const` just look polymorphic in return type or argument type

### Category of functors
* Morphisms are natural transformations
* Written category of functors between $C$ and $D$ is written $Fun(C, D)$, or $D^C$
    * Turns out category of small categories (**Cat**) is a cartesian closed category, which has an exponential for any pair of objects
    * Can identify that exponential with the functor category between two categories

### 2-Categories
* Generalization of category, where there are 2-morphisms (morphisms between morphisms)
* For **Cat**
    * Objects: (Small) categories
    * 1-morphisms: Functors
    * 2-morphisms: Natural transformations
* Instead of Hom-set, have Hom-category

### Problems
1. Define a natural transformation from the Maybe functor to the list functor. Prove the naturality condition for it.
    * ```Haskell
maybe_to_list :: Maybe a -> [a]
maybe_to_list Nothing = []
maybe_to_list (Just x)  = [x]
```

    * $Gf \circ \alpha_a = \alpha_b \circ Ff$
    * Want to show: `fmap f (maybe_to_list x) = maybe_to_list (fmap f x)`
    * If `x` is `Nothing`:
    * `fmap f (maybe_to_list x) = fmap f []`
    * `= []`
    * `= maybe_to_list Nothing`
    * `= maybe_to_list (fmap f Nothing)`
    * `= maybe_to_list (fmap f x)`
    * x = Just z:
    * `fmap f (maybe_to_list x) = fmap f [z]`
    * `= [f z]`
    * `= maybe_to_list (Just (f z))`
    * `= maybe_to_list (fmap f x)`

2. Define at least two different natural transformations between `Reader ()` and the list functor. How many different lists of `()` are there?
    * Two:
    ```Haskell
reader_to_list :: (() -> a) -> [a]
reader_to_list f = [f ()]
```
```Haskell
other_reader_to_list :: (() -> a) -> [a]
other_reader_to_list f = []
```

3. There are 3, one returning `Just (f False)`, one returning `Just (f True)`, and one returning `Nothing`.

4. Show that horizontal composition of natural transformation satisfies the naturality condition (hint: use components). It’s a good exercise in diagram chasing.
    * I gave it a shot

5. No

6. Create a few test cases for the opposite naturality condition of transformations between different Op functors.
    * Verify: `contramap f . alpha_a = alpha_b . contramap f`
    * Natural transformation:
    ```Haskell
    bool_op_to_string_op :: Op Bool a -> Op String a
    bool_op_to_string_op (Op f) = Op (\x -> if (f x) then "T" else "F")
```
    * Verification
    ```Haskell
    let first = contramap f (bool_op_to_string_op op)
    print (getOp first "3")
    let second = bool_op_to_string_op (contramap f op)
    print (getOp second "3")
```


## 2.2 Limits and Colimits
* Want to generalize universal construction of product
* Select objections $a$ and $b$ in $C$
    * Represent selection as functor $D$ from category **2** (2 objects, identity morphisms only) to $C$
* Next select candidate object $c$
    * Use constant functor $\Delta$ from **2** to $C$
    * $\Delta_c$ maps all elements of **2** to $c$, and all morphisms to $\text{id}_c$
* Now have two functors from **2** to $C$, so consider nautral transformations
* $D\: 1 = a$, $\Delta_c\: 1 = c$
    * Component of natural transformation from $\Delta_c$ to $D$ at 1 is a morphism $p : c \rightarrow a$
* $D\: 2 = b$, $\Delta_\c: 2 = c$
    * Component of nat. transform. at 2 is $q : c\rightarrow b$
* These two components are like the projections
* Naturality automatically satisfied as there are no non-identity morphisms in **2**
* In general, might be non-trivial morphisms, which constrain the transformation
* A transformation between $\Delta_c$ and $D$ is a _cone_ because the image of $\Delta$ is like the apex
* General case for building a cone:
    * start with category $I$
    * Pick $D$ from $I$ to $C$ and call it a _diagram_
    * Pick $\Delta_c$ from $I$ to $D$ to define apex $c$
    * Natural transformation from $\Delta_c$ to $D$ is the cone
    * Naturality requirement:
        * For any $f : a \rightarrow b$
        * $\alpha_b \circ \Delta_c\: f = D\: f \circ \alpha_a$
        * but $\Delta_c$ maps all morphisms to identity so
        * $\alpha_b = D\: f \circ \alpha_a$
* Now want to find _universal cone_
* Define category of cones, based on a functor $D$
    * Not all objects $c$ in $C$ can be apex, because there is no natural transformation between $\Delta_c$ and $D$
    * Need morphisms between cones, which would be full determined by morphisms between their apices
    * Generalization of factorization requirement for product is that exists unique $h: c \rightarrow \text{Lim}\: D$ such that for $g': c \rightarrow a$, $g: \text{Lim}\: D \rightarrow a$, $g' = g \circ h$
    * Take these factorizing morphisms as morphisms for category of cones
* Define universal cone as _terminal object_ in category of cones
* Call universal cone the _limit_ of diagram $D$
* Call apex _limit_ or _limit object_

### Limit as a natural isomorphisms
* Want to replace commutativity requirement for triangles linking cones with a naturality condition
* Given any cone, there's a unique factoring morphism from its apex, $c$, to the apex of the universal cone, $\text{Lim}\: D$
    * This gives a mapping of cones to morphisms
* Special morphism is a member of hom-set $C(c, \text{Lim}\: D)$
* Want to pick for each $c$, one morphisms from $C(c, \text{Lim}\: D)$
* Mapping of $c$ to $C(c, \text{Lim}\: D)$ is a functor from $C$ to **Set**
    * Contravariant
    * Map $c'$ to set $C(c', \text{Lim}\: D)$
    * Action on morphism $f : c' \rightarrow c$
    * Element of image of $c'$ is a morphism, $u : c \rightarrow \text{Lim}\: D$
    * $u \circ f : c' \text{Lim}\: D$, which is an element of $C(c', \text{Lim}\: D)$, so this gives us our mapping of morphisms
* Need another functor from $C$ to **Set**
* This time consider set of cones, i.e. the set of natural transformations $\text{Nat}(\Delta_c, D)$
* Map from $c$ to this set;
    * Take a morphism $f : c' \rightarrow c$
    * Lifted $f$ should map $\text{Nat}(\Delta_c, D) \rightarrow \text{Nat}(\Delta_{c'}, D)$ (because contravariant)
    * Consder $\alpha \in \text{Nat}(\Delta_c, D)$
    * $\alpha_a : \Delta_c a \rightarrow D\: a$
    * $\alpha_a : c \rightarrow D \: a$
    * Need to use $f$ and $\alpha$ to construct $\beta \in \text{Nat}(\Delta_{c'}, D)$, s.t.
    * $\beta_a : c' \rightarrow D\: a$
    * Just use $\beta_a = \alpha_a \circ f$
* So now have two contravariant functors from $C$ to **Set**
    * Functors from $C$ to **Set** are called presheaves
    * First functor defined is _representable presheaf_
* Now functor $D$ from $I$ to $C$ has a limit $\text{Lim}\: D$ if and only if there is a natural isomorphism between the two functors:
    * $C(c, \text{Lim}\: D) \simeq \text{Nat}(\Delta_c, D)$
    * i.e. natural transformation where every component is an isomorphism        

### Examples of Limits
* Categorical product is limit of diagram generated by **2**
* Terminal object is limit generated by empty category
    * Functors from empty category select no object, so cone is only apex
* An _equalizer_ is limit generated by two element category with two parallel morphisms going between them
    * Diagram in $C$ consists of:
    * Two objects $a$ and $b$ in $C$
    * Two morphisms `f :: a -> b`, `g :: a -> b`
    * Apex has projections `p :: c-> a`, `q :: c -> b`
    * Need `q = f . p`, and `q = g . p`
    * So `f . p = g . p`
    * In **Set**, image of `p` selects a subset of `a` such that the images of `f` and `g` are equal on that set
* A _pullback_ starts from three object category with the shape `1->2<-3`
    * Diagram is often called a _cospan_
    * Apex $d$
    * Projectoins `p :: d -> a`, `q :: d -> c`, `r :: d -> b`
    * Images of morphsims from original cateogry `f :: a -> b`, `g :: c -> b`
    * Need `g . q = f . p = r` (can omit `r`)
    * In **Set**, think of $d$ as consisting pairs of elements $x\in a$ and $y \in c$ for which $f(x) = g(y)$
    * In programming languages with multiple inheritance:
        * Consider B and C inheriting from A, and D inheriting from B and C
        * If D is a pullback, then any class E that inherits from B and C is also a cubclass of D
        * Casting E to D is safe if D is barebones combination of B and C with no additional data/overloading
    * Another type inference example, the need to _unify_ types:
        * Want to infer type of `twice f x = f (f x)`
        * Assign preliminary types:
            * `f :: t0`, `x :: t1`, `f x :: t2`, `f (f x) :: t3`
            * `twice :: t0 -> t0 -> t1 -> t3`
            * Apply constraints:
            * `t0 = t1 -> t2`, `t0 = t2 -> t3`
            * Need to unify constraints by finding types such that putting them into both constraints gives same type
            * Could do `t1 = t2 = t3 = Int`, but this is not most general solution
            * Most general solution is obtained via pullback

### Colimits
* Invert direction of all arrows in a cone to get a co-cone
* Initial object is colimit corresponding to diagram base don empty category
* Dual of pullback is called _pushout_
    * Based on diagram called span, generated by `1<-2->3`
    
### Continuity
* Functors come close to idea of continuity by not breaking existing connections
* A _continuous functor_ `F` from `C` to `C'` must also preserve limits
* So given `D` from `I` to `C` having a limit `Lim D`, `F . D` also must have a limit, and it must be equal to `F (Lim D)`
* Functors preserve cones, commuting triangles, and factorization
* Issue is uniqueness, there may be "better cones" in `C'` that weren't in `C`
* Hom-functors are continuous
    * Hom-functor `C(a, b)` is contravariant in first var, covariant in second
    * Functor which maps $C^{\text{op}} \times C \rightarrow \textbf{Set}$
    * Fixing second argument maps colimits in $C$ to limits in **Set**
    * Fixing first argument maps limits to limits
* In Haskell, hom-functor maps two types to a function type
    * e.g. `newtype ToString a = ToString (a -> String)`
    * Continuity 

### Problems
1. How would you describe a pushout in the category of C++ classes?
    * I believe that the push out of the bottom of a diamond is the most recent common ancestor of the two parent classes
2. Show that the limit of the identity functor Id :: C -> C is the initial object.
    * Consider cone which is a natural transformation between $\Delta_{c}$ and $\text{Id}$, call components $\alpha$
    * Then for any $f : a \rightarrow b$ in $C$
    * Exists $g : c \rightarrow b$ such that $g = \text{Id}\:f \circ \alpha_a$
    * Because all $b$s are on the receiving end of at least one morphism ($\text{Id}_b$), there is a morphism from $c$ to every $b$
    * So every cone apex has a morphism to every object in $C$
    * let $c$ be initial object
    * Let $f : c \rightarrow a$ be the single morphism connecting $c$ and $a$
    * Let $\alpha$ be edges of $c'$ cone
    * $f \circ \alpha_c = \alpha_a$ by naturality
    * But this means that $\alpha_c$ is the factorizer, so $c$ is the limit of $\text{Id}$

3. Subsets of a given set form a category. A morphism in that category is defined to be an arrow connecting two sets if the first is the subset of the second. What is a pullback of two sets in such a category? What’s a pushout? What are the initial and terminal objects?
    * Initial object is empty set
    * Terminal object is entire set
    * Pullback is intersection? (why isn't this product)
    * Pushout is union? (why isn't this coproduct)
    
4. Can you guess what a coequalizer is?
    * Had to look this up, but given maps $f$ and $g$ between $X$ and $Y$ in **Set**, the co-equalizer is a set of equivalence classes on $Y$ for the equivalence relation $\sim$ satisfying:
        * $f(x) \sim g(x)$
5. Show that, in a category with a terminal object, a pullback towards the terminal object is a product.
    * Pullback is `1->2<-3`
    * Call terminal object $t$, other two objects $a$ and $b$
    * Consider pullback $p$ (with cone edges $f : p\rightarrow a$ and $g: p\rightarrow b$) and candidate product $p'$ (of $a$ and $b$)
    
    * Candidate product $p'$ is also a candidate for pullback towards the terminal object because there is an arrow $p' \rightarrow t$, as well as arrows $f': p' \rightarrow a$, $g': p' \rightarrow b$, $a \rightarrow t$, and $b \rightarrow t$
    * So there exists a factoring arrow $h : p' \rightarrow p$ such that $f' = f \circ h$, and $g' = g\circ h$
    * So $p$ is also a better product than $p'$
6. Similarly, show that a pushout from an initial object (if one exists) is the coproduct.
    * Pushout is `1<-2->3`
    * Call initial object $i$, and let other two objects be $a$ and $b$
    * Consider pushout $p$ (with co-cone edges $f: a\rightarrow p$ and $g: b\rightarrow p$) and candidate coproduct $p'$ of $a$ and $b$
    * Candidate co-product $p'$ is also a candidate for pushout from initial object because there are arrows $i\rightarrow p'$, $f': p'\rightarrow a$, $g': p'\rightarrow b$, $i\rightarrow a$, and $i\rightarrow b$
    * So there exists a factoring arrow $h: p\rightarrow p'$ such that $f' = h \circ f$ and $g' = h\circ g$
    * So $p$ is a better coproduct than $p'$

## 2.3 Free Monoids
* Given initial set and identity $e$, generate all pairs
    * Identify $(a, e)$ and $(e, a)$ with $a$
* Repeat forever
    * Also identify products that should be equal by associativity
* If $e$ is the empty list, then this is equivalent to list concatentation
* e.g. `[Bool]` is the free monoid generated by `Bool`

### Free monoid universal construction
* **Mon** is category of monoids with homomorphisms as morphisms
* Every monoid is just a set, so can map monoids to sets and homomorphisms to functions
    * This is the _forgetful functor_
* Construction
    * Start with a set of generators $x$ in **Set**
    * Match pattern $p :: x \rightarrow U\: m$ where $U$ is forgetful functor from **Mon** to **Set**
    * $p$ identifies generators inside $U \: m$
    * Some functions might collapse some points, but the universal construction will handle that
    * Ranking candidates:
        * Consider another monoid $n$ and function $q :: x \rightarrow U\: n$ which identifies the generators in $n$'s image
        * $m$ is better than $n$ if there is a morphism in **Mon** $h::m\rightarrow n$ whose image under $U$ factorizes through $p$, i.e. $q = U\: h \circ p$
    * Morphism $h$ is where some elements can get identified with each other, so any other monoid with generators $x$ is just free monoid with some collapsed elements

### Problems
1. You might think (as I did, originally) that the requirement that a homomorphism of monoids preserve the unit is redundant. After all, we know that for all `a`

 `h a * h e = h (a * e) = h a`  

 So `h e` acts like a right unit (and, by analogy, as a left unit). The problem is that `h a`, for all `a` might only cover a sub-monoid of the target monoid. There may be a “true” unit outside of the image of `h`. Show that an isomorphism between monoids that preserves multiplication must automatically preserve unit.
    * Let $f$ be an isomorphism of monoids between $M$ and $N$ having units $e_M$ and $e_N$ respectively
    * For any $y \in N$, let $x$ be the preimage of $y$ under $f$
    * $y = f(x) = f(e_M \times_M x) = f(e_M) \times_N f(x) = f(e_M) \times_N y$
    * Same sort of thing for left multiplication by the unit
    * So $f(e_M) = e_N$
    
2. Consider a monoid homomorphism from lists of integers with concatenation to integers with multiplication. What is the image of the empty list `[]`? Assume that all singleton lists are mapped to the integers they contain, that is `[3]` is mapped to `3`, etc. What’s the image of `[1, 2, 3, 4]`? How many different lists map to the integer 12? Is there any other homomorphism between the two monoids?
    * Lists map to the product of their elements, so:
    * The image of the empty list is 1 `[]` is the concatenative identity
    * The image of `[1,2,3,4]` is 24
    * An infinite number of lists map to 12, because they may contain an arbitrary number of 1s
    * Another example of a homomorphism is mapping a list of length $L$ to $2^L$
3. What is the free monoid generated by a one-element set? Can you see what it’s isomorphic to?
    * The free monoid generated by a one element set (call the element $a$) just consists of the identity and powers of $a$. It is isomorphic to the natural numbers with addition.

## 2.4 Representable Functors
### The Hom Functor
* Every category has canonical mapping to **Set**
    * Fix $a$ in $C$ and pick $x \in C$
    * Hom-set $C(a, x)$ is a set in **Set**
    * Vary $x$, keeping $a$ fixed, $C(a, x)$ varies in **Set**
    * Maps morphisms as follows:
        * $x$ maps to $C(a, x)$ and $y$ maps to $C(a, y)$, so
        * $f$ must be mapped to a function $C(a, x) \rightarrow C(a, y)$
        * Will define function pointwise, so consider $h \in C(a, x)$
        * $f \circ h :: a\rightarrow y$ is in $C(a, y)$
        * Write this function as $C(a, f)$, and its action on $h$ as $C(a, f) h = f \circ h$
    * This is the `Reader` functor in Haskell `type Reader a x = a -> x`
* Now consider fixing second argument: $C(-, a)$ is a contravariant functor
* Called `Op` in earlier chapters
* Varying both, get profunctor
* $C(-, =) :: C^{\text{op}} \times C \rightarrow \textbf{Set}$


### Representable Functors
* For every $a$ in $C$, get a functor from $C$ to **Set**
* Preserves structure, so called a _representation_
* Objects in morphisms in $C$ are represented as sets and functions
* $C(a, -)$ is sometimes called representable, as is any functor $F$ which is naturally isomorphic to the hom-functor for some $a$
* Generally think of isomorphic sets as identical, and more generally think of isomorphic objects as identical
* Since functors form a category with natural transformations as morphisms, can think of two isomorphic functors as identical
* For a functor $F$ to be representable, require that there is an object $a\in C$, and natural transformations $\alpha : C(a, -) \rightarrow F$ and $\beta: F \rightarrow C(a, -)$ such that their composition is the identity
* The component of $\alpha$ at some object $x$ is:
    * $\alpha_x : C(a, x) \rightarrow F\: x$, which is a function in **Set**
    * Naturality tells us that for any $f: x \rightarrow y$
        * $F\: f \circ \alpha_x = \alpha_y \circ C(a, f)$
    * In Haskell: `fmap f . alpha = alpha . fmap f`
        * Right fmap is fmap for `Reader`, so can say that for any $h$ in $C(a, x)$:
        * `fmap f (alpha h) = alpha (f . h)`
* $\beta$ goes opposite direction:
    * $\beta_x : F\: x \rightarrow C(a, x)$
    * Must have $\beta \circ \alpha = \alpha \circ \beta = \text{id}$
    * Later: There is always a natural transformation from $C(a, -)$ to any **Set**-value functor $F$ if $F\: a$ is non-empty (Yoneda's Lemma)

* Example: list functor, and `Int` as `a`
 ```haskell
 alpha :: forall x. (Int -> x) -> [x]
 alpha h = map h [12]
 ```
    * Naturality just says `map f (map h [12]) = map (f \circ h) [12]
    * Inverse can't exist since would have to go from list of type $x$ to a function returning $x$
        * Can't try returning head of list, as won't work for empty list
    * List functor is not representable
* If haskell functors are like containers, representable functors are like storing memoized results of function calls
    * Representing object, the type $a$ in $C(a, -)$ is the key type
* For example, a stream of values is representable
    * `data Stream x = Cons x (Stream x)`
    * Think of as memoized values of a function which takes `Integer` as an argument (technically natural numbers)
    * ($\alpha$ is `tabulate` and $\beta$ is `index`)
    ```haskell
    instance Representable Stream where
    type Rep Stream = Integer
    tabulate f = Cons (f 0) (tabulate (f . (+1)))
    index (Cons b bs) n = if n == 0 then b else index bs (n - 1)
    ```
* Representability of covariant functors is similar but use $C(-, a)$
* Hom-sets can be interanlly treated as exponential objects in cartesian closed categories
* Hom-set $C(a, x)$ is equivalent to $x^a$ and for representable $F$ can write: $-^a = F$
    * Formally taking "log" gives $a = \log F$
    * Log properties give that functors based on product types can be represented by sum types, and sum type functors aren't generally representable